<a href="https://colab.research.google.com/github/RushanUmerov/cookies/blob/main/%D0%98%D0%BD%D1%82%D0%B5%D0%BD%D1%81%D0%B8%D0%B2_GPT_%D0%A3%D0%BD%D0%B8%D0%B2%D0%B5%D1%80%D1%81%D0%B8%D1%82%D0%B5%D1%82_%D0%B8%D1%81%D0%BA%D1%83%D1%81%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE_%D0%B8%D0%BD%D1%82%D0%B5%D0%BB%D0%BB%D0%B5%D0%BA%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Установка библиотек. Сервисные функции
!pip install -q langchain==0.1.17 tiktoken==0.6.0 faiss-cpu==1.8.0 langchain-community==0.0.37 nltk==3.8.1 openai==1.30.1
!pip -q install gspread oauth2client

from openai import OpenAI, AsyncOpenAI
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
import pathlib
import subprocess
import tempfile
import ipywidgets as widgets
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import re

import os
import openai
import tiktoken
import re


class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

class GPT():
  def __init__(self):
    self.client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

  def load_search_indexes(self, url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text
    return self.create_embedding(text)

  def load_prompt(self,
                  url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text
    return f'{text}'


  def create_embedding(self, data):
    def num_tokens_from_string(string: str, encoding_name: str) -> int:
      """Returns the number of tokens in a text string."""
      encoding = tiktoken.get_encoding(encoding_name)
      num_tokens = len(encoding.encode(string))
      return num_tokens

    source_chunks = []
    splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=200)

    for chunk in splitter.split_text(data):
      source_chunks.append(Document(page_content=chunk, metadata={}))

    # Создание индексов документа
    search_index = FAISS.from_documents(source_chunks, OpenAIEmbeddings(), )

    count_token = num_tokens_from_string(' '.join([x.page_content for x in source_chunks]), "cl100k_base")
    print('\n ===========================================: ')
    print('Количество токенов в документе :', count_token)
    print('ЦЕНА запроса:', (int(count_token)*0.1)/1000000*90, ' рублей')
    return search_index

  def answer(self, system, topic, temp = 1):
    messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": topic}
      ]

    completion = self.client.chat.completions.create(
      model="gpt-3.5-turbo-0125",
      messages=messages,
      temperature=temp
      )

    return completion.choices[0].message.content


  def insert_newlines(self, text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return "\n".join(lines)


  def dialog(self):
    user = ''
    dialog = ''

    print(f'{bcolors.OKBLUE}{bcolors.BOLD}С чем связан ваш интерес к искусственному интеллекту?{bcolors.ENDC}')

    while user.lower() not in ['stop', 'exit', 'выход']:
      user = input('Клиент: ')
      if (user == 'Stop'): break

      dialog += '\n\n' + 'Клиент: ' + user
      add_dialog = gpt.answer(expert_prompt, user)

      dialog += '\n\n' + 'Менеджер: ' + add_dialog
      print(f'\n{bcolors.OKBLUE}{bcolors.BOLD}Менеджер:{bcolors.ENDC} {add_dialog}')
      report = gpt.answer(validation_prompt, dialog)
      answer = gpt.answer(action_prompt, report)

      print(f'\n{bcolors.OKGREEN}{bcolors.BOLD}Отчёт системы:\n {bcolors.ENDC}{report}')
      print(f'\n{bcolors.HEADER}{bcolors.BOLD}Менеджер: {bcolors.ENDC}{answer}\n\n')

    return dialog

  def answer_index(self, system, topic, search_index, temp = 1, verbose = 0):

    #Выборка документов по схожести с вопросом
    docs = search_index.similarity_search(topic, k=5)
    if (verbose): print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join(['\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if (verbose): print('message_content :\n ======================================== \n', message_content)

    messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": f"Используйте следующий контекст, чтобы ответить на вопрос в конце.\n{message_content}\n"
                        f"\nВопрос:\n{topic}"}
      ]

    completion = self.client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
    temperature=temp
    )
        # example token count from the function defined above
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f"{completion.usage.prompt_tokens} токенов использовано на промпт")
    if (verbose): print('\n ===========================================: ')
    if (verbose): print(f'{completion.usage.total_tokens} токенов использовано всего (промпт-ответ).')
    if (verbose): print('\n ===========================================: ')
    if (verbose): print('ЦЕНА запроса с ответом :', ((int(completion.usage.completion_tokens)*1.5)/1000000*90) + int(((completion.usage.prompt_tokens)*0.5)/1000000*90), ' рублей.')
    if (verbose): print('\n ===========================================: ')
    print('ОТВЕТ : \n', completion.choices[0].message.content)

    # return completion

  def get_chatgpt_ansver3(self, system, topic, search_index, temp = 1):

    messages = [
      {"role": "system", "content": system},
      {"role": "user", "content": topic}
      ]

    completion = self.client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=messages,
      temperature=temp
      )
    print('ОТВЕТ : \n', completion.choices[0].message.content)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# @title Установка ключа OpenAI
import os
import getpass
from IPython.display import display
from IPython.display import HTML, clear_output

os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")
gpt = GPT()

# Функция настройки стиля для переноса текста в выводе ячеек
def set_text_wrap_css():
    css = '''
    <style>
    pre {
        white-space: pre-wrap;
    }
    </style>
    '''
    display(HTML(css))

get_ipython().events.register('pre_run_cell', set_text_wrap_css)

Введите OpenAI API Key:··········


<h3><b> Маркетинг: ответы на вопросы</b></h3>

In [ ]:
#Все маркетинговые материалы
marketing_index = gpt.load_search_indexes('https://docs.google.com/document/d/1GL864OFbkaek5GUZmti8lcUNiK7HK-tyJXVM5pMCGLE')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(



 ===========================================: 
Количество токенов в документе : 281453
ЦЕНА запроса: 2.533077  рублей


In [ ]:
marketing_chat_promt = '''Ты менеджер поддержки в чате компании, компания продает курсы по AI.
У тебя есть большой документ со всеми материалами о продуктах компании.
Тебе задает вопрос клиент в чате, дай ему ответ, опираясь на документ.
И отвечай максимально точно по документу, не придумывай ничего от себя.
Документ с информацией для ответа клиенту: '''

In [ ]:
gpt.answer_index(
    marketing_chat_promt,
    'Расскажи про тариф Продвинутый',
    marketing_index
)

ОТВЕТ : 
 Тариф "Продвинутый" предлагает обширную программу обучения по теме искусственного интеллекта. Стоимость этого тарифа составляет 219 900 рублей, но при предзаказе вы можете приобрести его за 149 900 рублей. Тариф длится 7 месяцев, и включает в себя следующее:

- 32 занятия по нейронным сетям
- 2 года поддержки куратора
- Гарантию трудоустройства
- Реализацию AI проекта
- 12 месяцев стажировок
- Дополнительно: 36 продвинутых занятий по нейронным сетям и 25 бонусных занятий по различным курсам, таким как "Python для анализа данных", "Python-разработчик", "Машинное обучение", "Глубокая математика", "Terra AI", "Продажа AI проектов", "AI директор".

Этот тариф рекомендуется для студентов, которые хотят получить максимум информации и опыта, включая 4 стажировки по выбору в различных проектах. Deзакончив программу обучения, студент получает возможность трудоустройства на платформе Университета.


In [ ]:
gpt.answer_index(
    marketing_chat_promt,
    'Расскажи про тариф Продвинутый',
    marketing_index,
    verbose = 1
)


 ===========================================: 
message_content :
 
=====================* Тариф “Продвинутый” стоимость 155.742 рублей - 7.079 рублей в месяц. Первый платёж через 3 месяца.


                                                                              * Тариф “AI под ключ” стоимость 274.908 рублей - 12.496 рублей в месяц. Первый платёж через 3 месяца.


                                                                              * Тариф ChatGPT стоимость 149.904 рублей - 6 246 рублей в месяц.


                                                                              * Тариф “Основной” стоимость -  141 992 рубля - 6 454 рублей в месяц
Первый платеж через  3 месяца


                                                                              * Тариф “Основной” 
Стоимость первых 10 мест - 137.408 рублей - 6.246 рублей в месяц. Первый платеж через 3 месяца.
Вторые десять мест - 146.575 рублей - 6.663 рублей в месяц. Первый платеж через 3 месяца.
Основная стоимость

In [ ]:
marketing_chat_promt_logic = '''Ты менеджер поддержки в чате компании, компания продает курсы по AI.
Ты максимально логичный человек, который отвечает структурированно, коротко и фактами.
У тебя есть большой документ со всеми материалами о продуктах компании.
Тебе задает вопрос клиент в чате, дай ему ответ, опираясь на документ
И отвечай максимально точно по документу, не придумывай ничего от себя.
Документ с информацией для ответа клиенту: '''

In [ ]:
gpt.answer_index(
    marketing_chat_promt_logic,
    'Расскажи про тариф Продвинутый',
    marketing_index
)

ОТВЕТ : 
 Тариф "Продвинутый" представляет собой обширную обучающую программу AI, охватывающую следующие аспекты:

1. 32 занятия по нейронным сетям.
2. 2 года поддержки куратора.
3. Гарантию трудоустройства.
4. Реализацию AI проекта.
5. 12 месяцев стажировок.
6. Дополнительно предоставляются 36 продвинутых занятий по нейронным сетям и 25 бонусных занятий.
7. Набор курсов включает: "Python для анализа данных", "Python-разработчик", "Машинное обучение", "Глубокая математика", "Terra AI", "Продажа AI проектов", "AI директор".

Стоимость тарифа составляет 219 900 рублей, но при предзаказе его цена составляет 149 900 рублей. Тариф "Продвинутый" рекомендуется студентам, желающим получить обширные знания и пройти 4 стажировки в различных проектах. Также предусмотрена возможность прохождения данного тарифа за 33 000 рублей с рассрочкой на 3 года и ежемесячным платежом в размере 4 719 рублей.


In [ ]:
marketing_chat_promt_emotional = '''Ты менеджер поддержки в чате компании, компания продает курсы по AI.
Ты максимально эмоциональный человек, который старается всегда вдохновить собеседника темой AI и вдохновить на изменения.
У тебя есть большой документ со всеми материалами о продуктах компании.
Тебе задает вопрос клиент в чате, дай ему ответ, опираясь на документ, постарайся ответить так
Документ с информацией для ответа клиенту: '''

In [ ]:
gpt.answer_index(
    marketing_chat_promt_emotional,
    'Расскажи про тариф Продвинутый',
    marketing_index
)

ОТВЕТ : 
 Тариф "Продвинутый" представляет собой насыщенную программу обучения по AI, которая обеспечит вас всем необходимым для успешного старта в сфере искусственного интеллекта. Стоимость тарифа составляет 219 900 рублей, однако при предзаказе вы можете приобрести его за 149 900 рублей, а также получить 40 000 бонусов, которые можно использовать для дополнительных курсов и консультаций.

В рамках тарифа "Продвинутый" вы получите:

- 32 занятия по нейронным сетям;
- 12 месяцев стажировок;
- 2 года поддержки куратора;
- Гарантию трудоустройства;
- Реализацию AI проекта;
- Дополнительно 36 продвинутых занятий по нейронным сетям и 25 бонусных занятий;
- Широкий спектр курсов, включая "Python для анализа данных", "Python-разработчик", "Машинное обучение", "Глубокая математика", курс по Terra AI, курс "Продажа AI проектов" и курс "AI директор".

Этот тариф рекомендуется студентам, которые стремятся углубленно изучить тему искусственного интеллекта, пройти четыре стажировки в различных про

In [ ]:
gpt.answer_index(
    marketing_chat_promt,
    'Расскажи про стажировки',
    marketing_index
)

ОТВЕТ : 
 Программа стажировок в нашей компании предлагает уникальный опыт обучения, включая работу над реальными проектами, взаимодействие с клиентами и командную работу. В ходе трехмесячной стажировки студенты имеют возможность выбрать свое направление и работать над проектами в таких крупных компаниях, как Камаз, Рудн, Совкомбанк, Ростелеком и другие. Программа стажировки включает в себя индивидуальные учебные треки, еженедельные звонки с руководителем группы, а также общение в общем чате в Телеграме. 

Студенты также получают возможность развить навыки работы в команде, взаимодействия с заказчиками и руководством, что помогает им прокачать себя за короткий срок. После прохождения стажировки студенты получают ценный опыт, который может быть востребован на рынке труда. Для участия в программе стажировок для старичков требуется оплата 40 000 рублей за год, в ходе которого студенты пройдут через 4 стажировки в крупных компаниях.

Кроме того, стажировки включают возможность помощи в тру

In [ ]:
gpt.answer_index(
    marketing_chat_promt,
    'Расскажи про выкуп стоимости обучения',
    marketing_index
)

ОТВЕТ : 
 Программа выкупа стоимости обучения компании предполагает, что студент имеет возможность вернуть всю сумму, потраченную на обучение, при устройстве на работу в Университет. Фактически, студент возвращает стоимость обучения через заработную плату на новом месте работы. Гарантия трудоустройства закреплена в договоре обучения. У студента даже есть возможность приступить к работе на следующий день после защиты диплома. В текущий момент предоставляется 10 мест для выкупа стоимости обучения, из которых уже воспользовались более 150 студентов.

Система работает следующим образом: после завершения обучения выпускники стартуют на рынке труда с зарплатой в районе:
- 70.000-100.000 рублей в месяц для Junior уровня;
- 100.000 - 130.000 рублей в месяц для Middle уровня с опытом программирования до обучения.

На практике выпускники выполняют такие задачи, как создание фреймворков, контента и AI проектов. Например, они работают над фреймворками по генетическим алгоритмам, сегментации и obje

<h3><b> Генерация постов для тг</b></h3>

In [ ]:
#Только посты в телеграм
marketing_tg_post_index = gpt.load_search_indexes('https://docs.google.com/document/d/1SukTbi018v4H8D8Lt9YM9i3Zz_BjEWxNw7aw6T3euo4')


 ===========================================: 
Количество токенов в документе : 64775
ЦЕНА запроса: 0.582975  рублей


In [ ]:
marketing_tg_post_promt = '''Ты эксперт и спикер по AI, ты пишешь посты в телеграм канал, твоя задача делать вдохновляющие, но в то же время конкретные и полезные посты.
Компания, в которой ты работаешь, продает курсы по AI.
У тебя есть большой документ с десятками твоих предыдущих постов и информацией о продуктах компании.
Тебе задает тему твой руководитель, ты должен написать пост в телеграм канал на эту тему.
Бери информацию для поста точно по документу, не придумывай ничего от себя.
Ты должен сохранить стиль написания такой же, как в постах в документе.
Документ с информацией для ответа: '''

In [ ]:
gpt.answer_index(
    marketing_tg_post_promt,
    'Напиши пост про AutoML, почему надо отказываться от обычного написания нейронок и переходить на AutoML',
    marketing_tg_post_index
)

ОТВЕТ : 
 🚀 В мире AI наступает эпоха AutoML - технологии, которая полностью изменит работу разработчиков нейронных сетей в 2023-2024! 🤖

🌟 Почему вам стоит перейти на AutoML и забыть об утомительном написании нейронок вручную?

1️⃣ **Скорость и эффективность**: AutoML позволяет создавать нейронные сети в разы быстрее. Например, задачу классификации засыпающих водителей группа из 10 разработчиков решила за 7 дней с точностью 99.0%. В то же время, AutoKeras достиг точности 99.3% всего за 48 минут! Это нечто 😲

2️⃣ **Простота использования**: Забудьте о сложном написании кода. С AutoML вы можете создать нейронку просто поставив задачу на обучение и получить готовый результат уже утром! 🌅

3️⃣ **Тренд AI**: Технологии AutoML и Zero-code - будущее AI. Кто не успеет освоить их сейчас, отстанет завтра. Автоматическое машинное обучение продолжит развиваться в 2023, 2024, 2025, и быть в тренде - значит быть в курсе событий.

4️⃣ **Практичность**: Наши тимлиды и студенты уже успешно используют 

In [ ]:
gpt.answer_index(
    marketing_tg_post_promt,
    'Напиши пост про курс по интеграции в production, сделай пост продающим и подробно раскрой выгоды прохождения курса по пунктам',
    marketing_tg_post_index
)

ОТВЕТ : 
 🚀 Новый курс по интеграции в production - ключ к вашему успеху в мире AI! 🚀

✅ Почему стоит выбрать этот курс:
1️⃣ Реальный практический опыт - мы учим не просто создавать нейронные сети, а успешно интегрировать их в production. Этот навык ценят работодатели и заказчики проектов.
2️⃣ Увеличение ваших шансов на рынке труда - компании и клиенты ищут специалистов, способных довести проект до внедрения, и наш курс поможет вам стать таким специалистом.
3️⃣ Повышение вашей ценности - чем больше у вас компетенций, тем легче вам реализовывать проекты, находить работу и убеждать клиентов в своих навыках.

🔥 Что вас ждет на курсе:
📌 Практические навыки интеграции в production, которые сразу можно применить на практике.
📌 Глубокое погружение в различные случаи интеграции - от создания api для веб-сервиса до интеграции в мобильные приложения и конвейеры на производстве.
📌 Возможность учиться у профессионалов с многолетним опытом в области AI.

🚨 Важно! Даже если вы не ищете работу, эти н

<h3><b> Генерация заголовков таргетированной рекламы</b></h3>

In [ ]:
#Только страницы сайта
marketing_targeting_index = gpt.load_search_indexes('https://docs.google.com/document/d/1puW6Ckm2e5Kc59UKOsnVsHcNXgYlgftFaM4SjI57WdM')


 ===========================================: 
Количество токенов в документе : 5093
ЦЕНА запроса: 0.045836999999999996  рублей


In [ ]:
marketing_targeting_promt = '''Ты специалист настройки таргетированно рекламы, компания, в которой ты работаешь, продает курсы по AI.
У тебя есть большой документ с информацией о продуктах компании.
Твоя задача писать заголовки таргетированной рекламы с акцентами, который задаёт пользователь в запросе.
Бери информацию для заголовка рекламы точно по документу, не придумывай ничего от себя.
Документ с информацией для генерации заголовка: '''

In [ ]:
gpt.answer_index(
    marketing_targeting_promt,
    'Напиши 3 заголовка для таргетированной рекламы',
    marketing_targeting_index
)

ОТВЕТ : 
 1. "Стань AI разработчиком с нашим курсом по Data Science и нейронным сетям! Создавай нейронки без опыта программирования!"
2. "Узнайте все о компьютерном зрении, обработке естественного языка и других темах AI! Самая большая база контента в мире!"
3. "Присоединяйтесь к программе, одобренной Российской академией наук, и развивайте AI в России! Начните с нуля и зарабатывайте от 150.000р в месяц!"


In [ ]:
gpt.answer_index(
    marketing_targeting_promt,
    'Напиши 3 заголовка для таргетированной рекламы с упором на трудоустройство',
    marketing_targeting_index
)

ОТВЕТ : 
 1. "Гарантированное трудоустройство в IT: до 200 000р в месяц после обучения AI!"
2. "Станьте Data Scientist и получайте от 70 000р уже сразу после курса!"
3. "Уникальная возможность для программистов: трудоустройство с зарплатой до 200 000р в месяц!"


In [ ]:
gpt.answer_index(
    marketing_targeting_promt,
    'Напиши 3 заголовка для таргетированной рекламы с упором на создание AI проекта',
    marketing_targeting_index
)

ОТВЕТ : 
 1. Создавайте собственные AI проекты под руководством опытных кураторов!
2. Реализуйте свои идеи с помощью курса по Data Science и нейронным сетям!
3. Станьте Middle AI developer и создайте свой первый успешный проект с нами!


In [ ]:
gpt.answer_index(
    marketing_targeting_promt,
    'Напиши 3 заголовка для таргетированной рекламы с упором на то, что AI - это очень интересно',
    marketing_targeting_index
)

ОТВЕТ : 
 1. "Открой мир нейросетей: обучение AI с практиками искусственного интеллекта!"
2. "Увлекательное погружение в мир AI: создавай нейронные сети без опыта программирования!"
3. "AI для всех: станьте разработчиком и воплотите свои идеи в инновационные проекты!"


<h3><b>Менеджер поддержки HR отдела</b></h3>

In [ ]:
#HR база
hr_index = gpt.load_search_indexes('https://docs.google.com/document/d/1ZjpABLZD9BFDjPvYFcwOVcH5nnwx1lkpUCLneBUrzIQ')


 ===========================================: 
Количество токенов в документе : 13719
ЦЕНА запроса: 0.12347100000000001  рублей


In [ ]:
hr_promt = '''Ты менеджер поддержки hr отдела, к тебе могут обращаться hr менеджеры и рекрутеры за подсказками и ответами на их вопросы в чате компании.
Постарайся дать развернутый ответ, твоя задача ответить так, чтобы у менеджера не осталось больше вопросов к тебе.
Отвечай по существу, без лишних эмоций и слов, от тебя нужна только точная информация.
Отвечай максимально точно по документу, не придумывай ничего от себя.
Документ с информацией для ответа клиенту: '''

In [ ]:
gpt.answer_index(
    hr_promt,
    'Как выбрать заголовок вакансии',
    hr_index
)

ОТВЕТ : 
 Для выбора заголовка вакансии необходимо следовать данным инструкциям:
1. Придумайте 5-10 вариантов заголовков вакансий, содержащих различные ключевые слова, по которым кандидаты могли бы найти вакансию.
2. Зайдите на сайт поиска.
3. Введите первое название вакансии.
4. Настройте регион и специализацию для отсева неподходящих кандидатов.
5. Посмотрите количество резюме и соискателей, подходящих под параметры.
6. Откройте 5-10 резюме соискателей и оцените, насколько их опыт и обязанности соответствуют вашей вакансии.
7. Повторите процесс для каждого заголовка, пока не найдете название, удовлетворяющее обоим критериям: количество соискателей и их релевантность.
8. Если вакансия уже опубликована, а вы хотите сменить заголовок, посмотрите примеры названий на странице вакансии в разделе "Посмотреть резюме, подходящих к этой вакансии".



In [ ]:
gpt.answer_index(
    hr_promt,
    'Как оплатить базу кандидатов',
    hr_index
)

ОТВЕТ : 
 Для оплаты базы кандидатов на hh.ru вам необходимо выполнить следующие шаги:

1. Перейдите на страницу hh.ru, зайдите в свой аккаунт и выберите в меню "Прайс-лист" пункт "База резюме".
2. Выберите нужный тариф и нажмите на кнопку "Купить".
3. Выберите способ оплаты (банковская карта, электронные деньги или банковский перевод) и заполните все необходимые поля.
4. Подтвердите платеж и дождитесь подтверждения оплаты.
5. После успешной оплаты вам придет чек на оплату, который необходимо сохранить и передать ответственному за бюджеты/оплату компании.

После выполнения этих шагов вы получите доступ к базе резюме кандидатов и сможете скачивать резюме в различных форматах.


In [ ]:
gpt.answer_index(
    hr_promt,
    'Как настроить таргет hh',
    hr_index
)

ОТВЕТ : 
 Для настройки таргетированной рекламы на HH.ru, следуйте этим инструкциям:

1. Перейдите по ссылке для входа: https://clickme.hh.ru/new/dashboard/
2. Выберите критерии для таргетинга, такие как регион, отрасль, должность, навыки, уровень образования и опыт работы, которые наиболее важны для вашей компании и вакансии.
3. В графе «Места размещения» оставьте «Выбирать площадки автоматически».
4. Задайте общий бюджет для кампании (например, 9000) и дневной бюджет (например, 6000).
5. Нажмите «Запустить».
6. Отслеживайте результаты кампании, включая количество кликов и просмотров объявлений, чтобы определить эффективность и внести необходимые изменения.


In [ ]:
gpt.answer_index(
    hr_promt,
    'Для чего нужны шаблоны вакансий?',
    hr_index
)

ОТВЕТ : 
 Шаблоны вакансий нужны для ускорения процесса публикации новых вакансий и сохранения единообразия в оформлении. При использовании шаблонов вакансий на HH.ru вы можете создавать один раз общий шаблон вакансии с заполненной информацией о должности, требованиях к кандидату, условиях работы и зарплате, а затем использовать его для создания новых вакансий, редактировать его в любое время и сохранять в удобной форме.


<h3><b>Методичка: карьерный трек AI разработчика</b></h3>

In [ ]:
#Методичка
AI_track_index = gpt.load_search_indexes('https://docs.google.com/document/d/1brdjCZN4zoy_drQUISUAX0541HfGMwcx3jlyhsGoWF4/edit#heading=h.k1ya7on3o1s9')


 ===========================================: 
Количество токенов в документе : 19194
ЦЕНА запроса: 0.172746  рублей


In [ ]:
AI_track_promt = '''Ты методолог и преподаватель в компании, которая обучает AI разработчиков.
Перед тобой документ, в котором описаны варианты карьерного трека AI разработчика и методология подбора темы применения AI для компаний.
Твоя задача придумывать экзаменационные вопросы.
Тебе будут давать тему и твоя задача придумать несколько экзаменационных вопросов для студентов по этой теме исходя из документа.
Составляй вопросы максимально точно по документу, не придумывай ничего от себя.
Документ: '''

In [ ]:
gpt.answer_index(
    AI_track_promt,
    'Придумай 5 экзаменационных вопросов про подбор темы применения AI в компании',
    AI_track_index
)

ОТВЕТ : 
 1. Какие вопросы необходимо задать себе при выборе бизнес-процесса для применения AI?
2. Какие шаги следует предпринять для описания деталей выбранного бизнес-процесса?
3. Какова цель использования алгоритма для выбора AI-проекта в компании?
4. Какие типы данных из IT-систем можно использовать для применения AI в бизнес-процессах?
5. Почему важно выявить проблемные места в текущем бизнес-процессе при выборе темы для AI-проекта?


In [ ]:
gpt.answer_index(
    AI_track_promt,
    'Придумай 5 экзаменационных вопросов про карьерный трек AI разработчика',
    AI_track_index
)

ОТВЕТ : 
 1. Что включает в себя первый этап карьерного трека AI разработчика, и почему важно определить свою текущую профессию?
2. Почему важно знать ваш текущий уровень дохода при формировании карьерного трека в AI-разработке?
3. Как участие в хакатонах и соревнованиях Kaggle может повлиять на вашу карьеру в AI-разработке и почему рекомендуется участвовать в них?
4. Как стажировка может оказать влияние на вашу карьеру в AI-разработке, и почему стажировка важна для развития как специалиста?
5. Почему писание статей и преподавание могут быть важными аспектами карьерного роста AI разработчика, и какие преимущества они могут принести?


In [ ]:
gpt.answer_index(
    AI_track_promt,
    'Придумай 5 экзаменационных вопросов про виды AI задач',
    AI_track_index
)

ОТВЕТ : 
 1. Какие типовые вопросы могут быть решены при помощи распознавания речи?
2. Чем отличаются типовые и не типовые вопросы в контексте создания сложного чат-бота?
3. Какие данные могут быть распознаны и сопоставлены с лицом пассажира при использовании AI в процессе проверки билетов и паспортов?
4. Какие шаги могут быть автоматизированы с помощью AI в процессе обработки покраски банок на конвейере?
5. Зачем необходим датасет для реализации AI-проекта и как он может быть использован при применении нейронных сетей?


<h3><b>Саммаризация zoom презентации менеджера</b></h3>

In [ ]:
#Методичка
zoom_index = gpt.load_search_indexes('https://docs.google.com/document/d/1MuQ02a3Kz6ysDN43SM5YrURS2Mg2trLNrM_TyIElZLQ')


 ===========================================: 
Количество токенов в документе : 20269
ЦЕНА запроса: 0.182421  рублей


In [ ]:
zoom_promt = '''Инструкция по оценке диалога менеджера по продажам с клиентом

Перед тобой диалог менеджера по продажам с клиентом
Тебе надо проверить несколько критериев и заполнить отчёт
Не пиши общее сообщение, только отчёт по форме, форма отчёта будет в конце

Что надо проверить
1. Говорил ли клиент о потребности в трудоустройстве
2. Говорил ли клиент о потребности создать AI проект
3. Говорил ли клиент возражение о стоимости обучения
4. Говорил ли клиент возражение о времени на обучение

Форма отчёта
1. Потребность о трудоустройстве - было или нет
2. Потребность о создании AI проекте - было или нет
3. Возражение о стоимости - было или нет
4. Возражение о времени обучения - было или нет

Заполни отчёт и пришли в качестве ответа, коротко и ёмко'''

In [ ]:
gpt.answer_index(
    zoom_promt,
    'Пришли отчет',
    zoom_index
)

ОТВЕТ : 
 1. Потребность о трудоустройстве - было
2. Потребность о создании AI проекте - не было
3. Возражение о стоимости - не было
4. Возражение о времени обучения - не было


In [ ]:
zoom_promt_2 = '''Ты менеджер контроля качества, твоя задача анализировать диалоги менеджеров по продажам с клиентами и готовить отчеты.
Компания продает курсы по AI разработке.
Перед тобой текст диалога сделанный с помощью распознавания речи из записи zoom презентации.
Из-за машинного распознавания речи, в тексте могут быть ошибки распознавания, учитывая это.
Твоя задача делать отчеты по данному диалогу по запросам пользователя.
Составляй вопросы максимально точно по диалогу, не придумывай ничего от себя.
Текст диалога: '''

In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Напиши отчет, какие были потребности названы клиентом',
    zoom_index
)

ОТВЕТ : 
 Отчет:

Клиент выразил интерес к созданию персонального шедевра в области рукоделия, чтобы определить свой стиль, разобраться в модных тенденциях, подобрать уникальное изделие и сделать его без предварительного обучения. Он выразил желание создавать что-то, что точно подойдет ему, максимально модно, возможно с оптимизацией по некоторым параметрам. Клиент также упомянул о боте в медицине, который может находиться в разработке.


In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Напиши отчет, почему клиенту вообще интересно изучение AI',
    zoom_index
)

ОТВЕТ : 
 Отчет:

Клиенту интересно изучение искусственного интеллекта из-за его модернизации и того факта, что она описывает как что-то современное, что привлекает ее внимание. Она также рассказывает о своем опыте программирования, упоминая о том, что в прошлом занималась программированием и у нее есть диплом по языку программирования Паскаль. Ее старший ребенок также учится на программиста, что подтверждает интерес и связь с темой. Клиент выражает желание рассмотреть обучение более подробно, что свидетельствует о ее заинтересованности и готовности углубить знания в области искусственного интеллекта.


In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Напиши отчет, какие возражения назвал клиент',
    zoom_index
)

ОТВЕТ : 
 Отчет:

Клиент выразил следующие возражения в ходе диалога с менеджером:
1. У клиента есть сомнения по поводу важности вопроса об обучении.
2. Клиент отметил, что долгое время занималась домашними делами и имеет сомнения относительно своей способности заниматься обучением.
3. Клиент интересуется возможностью начать обучение с базы и выражает сомнения по поводу выбора тарифа.
4. Клиент упомянул, что было время, когда занималась программированием, но это было давно, и она не уверена в своих способностях.
5. Клиент считает, что необходимо хорошо подумать перед началом обучения и интересуется возможностью пробного занятия.
6. Клиент выразил сомнения относительно трудоустройства и интересовалась возможностью заработка на проектах в дополнение к основной деятельности.

Это основные возражения, которые были названы клиентом в ходе диалога с менеджером.


In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Напиши отчет, насколько проактивно менеджер действовал в диалоге',
    zoom_index
)

ОТВЕТ : 
 Отчет:

Менеджер был довольно проактивным в этом диалоге. Он не только инициировал звонок с клиентом, но также активно задавал вопросы и предлагал конкретные варианты решения для клиента. Менеджер привлек внимание клиента к презентации, предложил различные вакансии для выбора, обсудил доступное время для обучения, а также детально рассказал о программах обучения и проектах, в которых студенты могут участвовать. Он также подчеркнул значимость опыта студентов в реальных проектах и возможность трудоустройства по завершении обучения. Менеджер был внимателен к клиенту и стремился предложить наиболее подходящие варианты для его обучения и развития в сфере искусственного интеллекта.


In [ ]:
gpt.answer_index(
    zoom_promt_2,
    'Напиши отчет, в котором будут цитаты из текста, которые говорят о потребностях клиента',
    zoom_index
)

ОТВЕТ : 
 Отчет:

1. Клиент выразил желание заниматься не менее четырех часов в неделю: "Есть время. Занимаетесь, главное, чтобы не менее четырех часов в неделю."
2. Клиент интересуется возможностью прохождения обучения на своем уровне сложности: "Формат. У нас есть текст. Вы занятие записи да чат с куратором. Он также прямо на платформе."
3. Клиент интересуется гибкостью и возможностью заморозки обучения: "Вдруг там, если хотите подохнуть, а там свидетельство не что, то такое на два месяца заморозили."
4. Клиент хочет пройти обучение быстрее, в течение двух с половиной месяцев: "Допустим, там у нас самый короткий срок. Уже два с половиной месяца, полное обучение."


<h3><b>Ответы на вопросы студентов</b></h3>

In [ ]:
#Методичка
conv_index = gpt.load_search_indexes('https://docs.google.com/document/d/1OzZvgAKPPozpin_D4g7UMC74US5yZnWznZRcQYGs2yM')


 ===========================================: 
Количество токенов в документе : 69165
ЦЕНА запроса: 0.622485  рублей


In [ ]:
conv_promt = '''Ты преподаватель по AI и опытный AI разработчик.
Ты умеешь объяснить все простыми словами для новичков.
Тебе будут писать новички, которые изучают AI.
И у тебя есть текст документа - это занятие по сверточным нейронным сетям.
Документ взят из google colab, в нем есть и текст, и исходники кода на python.
Ответь на вопросы обучающегося.
Составляй ответы точно по документу, не придумывай ничего от себя.
Документ: '''

In [ ]:
gpt.answer_index(
    conv_promt,
    '''Для чего мы используем в этом коде запись {:<20}?
      print('Класс: {:<20} {:3.0f}% сеть отнесла к классу {:<20} - {}'.format(class_labels[cls],
                                                                               100. * cm[cls, cls_pred],
                                                                               class_labels[cls_pred],
                                                                               msg))''',
    conv_index
)

ОТВЕТ : 
 В этом коде запись {:<20} используется для выравнивания строки на 20 символов слева. То есть, если длина строки, подставленной вместо {:<20}, меньше 20 символов, то к этой строке добавляются пробелы справа, чтобы строка занимала 20 символов.


In [ ]:
gpt.answer_index(
    conv_promt,
    '''Что здесь значит CLASS_LIST?
      compile_train_eval_model(model_conv,
                     	train_generator,
                     	validation_generator,
                     	test_generator,
                     	class_labels=CLASS_LIST,
                     	title='Сверточный классификатор')''',
    conv_index
)

ОТВЕТ : 
 CLASS_LIST - это список классов, к которым относятся изображения в нашем датасете. В данном случае CLASS_LIST содержит три класса: 'Ferrari', 'Mercedes' и 'Renault'. При обучении модели сверточной нейронной сети, параметр class_labels=CLASS_LIST указывает на передачу этого списка классов для корректной идентификации меток классов при оценке работы модели на тестовых данных.
